# Cutout with twin color pairs  multi colors

- author Sylvie Dagoret-Campagne
- creation date 2024-06-21
- last update 2024-06-22
- affiliation : IJCLab
- Kernel **w_2024_16**

- 1) Start by two-objects-lists sources in different bands. Thowe object are close enough to expect their visit in same calexp frequently
- 2) **Merge above visits lists**  in same band/same calexp for object 1 and Object 2 : df_g, df_r, df_z, df_y
- 3) Find closest time visit in other bands : dfg_r,

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
import scipy

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle
import lsst.sphgeom

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146' # test this one today 2024/06/22

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

## Definition of functions used

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
def find_neighbourg(ra0,dec0,src):
    """ 
    Calculate the angular separation of a list of sources in src pandas table from a (ra0,dec0) source location
    """
    all_angles = []
    for index, row in src.iterrows():
        spherePointCenter = lsst.geom.SpherePoint(ra0*lsst.geom.degrees, dec0*lsst.geom.degrees)
        spt = lsst.geom.SpherePoint(row["ra"]*lsst.geom.degrees, row["dec"]*lsst.geom.degrees)
        ang = spherePointCenter.separation(spt)
        ang_arcsec = ang.asArcseconds()
        all_angles.append(ang_arcsec)
    return all_angles

In [ ]:
def PlotAndFitHisto(data,std_clip,ax,nbins=50,xlabel=r'$(G-R)_1 - (G-R)_2$ :',title="Differential color with 2 stars",xmin=-100.,xmax=100.):
    #data = df_col_gr["ratio3"].values

    hist, bin_edges = np.histogram(data,bins=nbins,range=(xmin,xmax),density=True)
    bin_centres = (bin_edges[:-1] + bin_edges[1:])/2

    # Define model function to be used to fit to the data above:
    def gauss(x, *p):
        A, mu, sigma = p
        return A*np.exp(-(x-mu)**2/(2.*sigma**2))

    # p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
    p0 = [1., 0., 1.]

    coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)

    # Get the fitted curve
    hist_fit = gauss(bin_centres, *coeff)

    textstr = '\n'.join((
    xlabel,
    r'$\sigma_{clipped} = %.01f$ mmag' %(std_clip,),
    r'$\sigma_{fit} = %.01f$ mmag' %(coeff[2], )))

    ax.step(bin_centres, hist,'k' ,label='data')
    ax.plot(bin_centres, hist_fit,'r-', label='fit')
    ax.set_xlabel(xlabel)
    ax.legend()
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="blue", weight='bold',bbox=props)
    ax.set_title(title)
    ax.set_xlim(xmin,xmax)
    # Finally, lets get the fitting parameters, i.e. the mean and standard deviation:
    #print('Fitted mean = ', coeff[1])
    #print('Fitted standard deviation = ', coeff[2])

In [ ]:
def closestvisit(df1,df2):
    """
    This function associate pairs of closest observation

    input : the 2 dataframes (first to one for which one want to find the closest obs in dtaframe 2)
    output : the dataframe of similar format as the first one having columns on nightobs , visit time, dt from second dataframe)
    """
    d1 =df1.copy(deep=True)
    d2 =df2.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["nightObs2","visit2","time2","deltat"])
    for key in d1.index:

        nightObs = key[0]
        time1 = d1.loc[key]["time"]
        d2["dt"] = np.abs(d2["time"]-time1)
        d2["Dt"] = d2["time"]-time1
        cut_sel = (d2["dt"] == d2["dt"].min()) #& (d2["nightObs"] == nightObs)
        sample = d2[cut_sel]
        #df.iloc[key] = [sample.index[0][0],sample.index[0][1],sample["time"].values[0],sample['dt'].values[0]]
        df.loc[key]["nightObs2"] = sample.index[0][0]
        df.loc[key]["visit2"] = sample.index[0][1]
        df.loc[key]["time2"] = sample["time"].values[0]
        df.loc[key]["deltat"] = pd.to_timedelta(sample["Dt"].values[0], unit="s").total_seconds()
        all_samples.append(sample)

    # select observations during the same night
    df_back = df.copy(deep=True)
    flag_sel = []
    for key in df_back.index:
        nightObs = key[0]
        nightObs2 = df_back.loc[key]["nightObs2"]
        flag_sel.append(nightObs == nightObs2)
    df = df[flag_sel]
    return df

In [ ]:
def getcutoutandimage(visitId,df_selectedvisits):
    """
    visitId : the calexp where are the two stars
    df_selectedvisits : the list of all sources and visit of the main targeted star
    """

    row_source = df_selectedvisits[df_selectedvisits.visit == visitId].iloc[0]  
    band = row_source['band']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)   
    src_cat = butler.get('sourceTable',**dataId, collections=collections)

    #select on star
    #src_cat = src_cat[(src_cat["extendedness"]==0) & (src_cat["parentSourceId"] == 0) ]
    #src_cat = src_cat[src_cat["extendedness"]==0 ]

    x1 = row_source['x_x']
    y1 = row_source['y_x']
    ra1= row_source['ra_x']
    dec1= row_source['dec_x']
    psfMag1  = row_source['psfMag_x']  
    psfSigma1 = row_source['psfSigma_x']
    psfMagDiffMmag1 = row_source['psfMagDiffMmag_x'] 
    apFlux_50_0_instFlux1 = row_source['apFlux_50_0_instFlux_x']
    apFlux_50_0_instFluxErr1 = row_source['apFlux_50_0_instFluxErr_x']


    x2 = row_source['x_y']
    y2 = row_source['y_y']
    ra2= row_source['ra_y']
    dec2= row_source['dec_y']
    psfMag2  = row_source['psfMag_y']  
    psfSigma2 = row_source['psfSigma_y']
    psfMagDiffMmag2 = row_source['psfMagDiffMmag_y'] 
    apFlux_50_0_instFlux2 = row_source['apFlux_50_0_instFlux_y']
    apFlux_50_0_instFluxErr2 = row_source['apFlux_50_0_instFluxErr_y']

    
    # searcch the neighbouring stars
    #separation = find_neighbourg(ra_target,dec_target,src_cat)
    #src_cat["sep"] = separation


    spherePointCenter = lsst.geom.SpherePoint(ra1*lsst.geom.degrees, dec1*lsst.geom.degrees)
    spt = lsst.geom.SpherePoint(ra2*lsst.geom.degrees, dec2*lsst.geom.degrees)
    ang = spherePointCenter.separation(spt)
    ang_arcsec = ang.asArcseconds()
    
   
    x0 = (x1+x2)/2.
    y0 = (y1+y2)/2.
    ra0 = (ra1+ra2)/2.
    dec0 = (dec1+dec2)/2.
    

    dx1 = x1-x0
    dy1 = y1-y0
    dx2 = x2-x0
    dy2 = y2-y0
    
    
    boxSize = int(max(np.abs(x1-x2),np.abs(y1-y2)))
    xmin= x0-boxSize
    xmax= x0+boxSize
    ymin= y0-boxSize
    ymax= y0+boxSize
        
    
    minBbox = geom.Point2I(int(x0) - boxSize ,int(y0) - boxSize)
    maxBbox = geom.Point2I(int(x0) + boxSize, int(y0) + boxSize)
    
    srcBbox = geom.Box2I(minBbox, maxBbox)


    row_source['x0'] = x0
    row_source['y0'] = y0
    row_source['xmin'] = xmin
    row_source['ymin'] = ymin
    row_source['xmax'] = xmax
    row_source['ymax'] = ymax
    row_source['ra0'] = ra0
    row_source['dec0'] = dec0

    row_source['x1'] = x1
    row_source['y1'] = y1
    row_source['dx1'] = dx1
    row_source['dy1'] = dy1
    row_source['ra1'] = ra1
    row_source['dec1'] = dec1

    row_source['x2'] = x2
    row_source['y2'] = y2
    row_source['dx2'] = dx2
    row_source['dy2'] = dy2
    row_source['ra2'] = ra2
    row_source['dec2'] = dec2
    
    
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        #subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        #subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        cutout=subimg.image.array
        
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        cutout = None

    df_pair = row_source.to_frame()
    return row_source,cutout



In [ ]:
def plotfullcalexp(row_pair,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(10,10))
    else:
        flag_ax = True

    band = 0
    ra = 0
    dec = 0

    x0 = row_pair["x0"]
    y0 = row_pair["y0"]
    
    visitId = row_pair["visit"]
    band = row_pair["band"]
    psfMag1 = row_pair["psfMag_x"]
    psfSigma1 = row_pair["psfSigma_x"]
    x1 = row_pair["x1"]
    y1 = row_pair["y1"]
    ra1 = row_pair['ra2']
    dec1 = row_pair['dec2']
    ann1 = Annulus((x1,y1),r=5*psfSigma1,width=0.1,color="r")
    ax.add_patch(ann1)

  
    psfMag2 = row_pair["psfMag_y"]
    psfSigma2 = row_pair["psfSigma_y"]
    x2 = row_pair["x2"]
    y2 = row_pair["y2"]
    ra2 = row_pair['ra2']
    dec2 = row_pair['dec2']
    ann2 = Annulus((x2,y2),r=5*psfSigma2,width=0.1,color="orange")
    ax.add_patch(ann2)

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    imgarr = calexp.image.array 
    shape =  imgarr.shape
    #extent = [x0-shape[1]/2., x0+shape[1]/2., y0-shape[0]/2., y0+shape[0]/2.]
   
    z = ZScaleInterval()
    z1,z2 = z.get_limits(imgarr)
    im = ax.imshow(imgarr,origin="lower",cmap="grey",vmin=z1, vmax=z2)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    plt.colorbar(im, cax=cax)

    
    title = f"calexp {visitId}, objidx:{objectname1},{objectname2} band :{band}"
    ax.set_title(title)
    ax.axvline(x0,color="w",linestyle=":")
    ax.axhline(y0,color="w",linestyle=":")
    

    textstr = '\n'.join((
    r'$exposure = %.0f$' % (visitId, ),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$ra_1 = %f deg $' %(ra1,),
    r'$ra_2 = %f deg $' %(ra2,),  
    r'$dec_1 = %f deg $' %(dec1,),
    r'$dec_2 = %f deg $' %(dec2,)))  
  

    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="y", weight='bold',bbox=props) 
    #if flag_deblend_skipped:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="r", weight='bold',bbox=props) 
    #else:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="g", weight='bold',bbox=props) 

    if not flag_ax:
        plt.show()   

In [ ]:
def plotcutout(row_pair,cutout,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
    else:
        flag_ax = True

    if cutout is None:
        print("No Cutout")
        return

    boxSize= cutout.shape[0]//2
    extent = [ row_pair['xmin'], row_pair['xmax'], row_pair['ymin'], row_pair['ymax']]
    
    z = ZScaleInterval()
    z1,z2 = z.get_limits(cutout)
    im = ax.imshow(cutout,origin='lower',cmap="grey",vmin=z1, vmax=z2,extent = extent)
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    #plt.colorbar(im, cax=cax)

  

    band = 0
    ra = 0
    dec = 0
    
  
    visitId = row_pair["visit"]
    band = row_pair["band"]
    psfMag1 = row_pair["psfMag_x"]
    psfSigma1 = row_pair["psfSigma_x"]
    x1 = row_pair["x1"]
    y1 = row_pair["y1"]
    ra1 = row_pair['ra1']
    dec1 = row_pair['dec1']
    ann1 = Annulus((x1,y1),r=3*psfSigma1,width=0.1,color="r")
    ax.add_patch(ann1)

  
    psfMag2 = row_pair["psfMag_y"]
    psfSigma2 = row_pair["psfSigma_y"]
    x2 = row_pair["x2"]
    y2 = row_pair["y2"]
    
    ann2 = Annulus((x2,y2),r=3*psfSigma2,width=0.1,color="orange")
    ax.add_patch(ann2)
    
    #title = f"obj:{objectname}, b:{band}, (ra:{ra:.2f}, dec:{dec:.2f})"
    #ax.set_title(title)

    textstr = '\n'.join((
    r'$expos = %.0f$' % (visitId, ),
    r'$band  = %s $' % (band,),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$\sigma_{PSF} = %.2f pix$' % (psfSigma1 , )))
   
  
    ax.text(0.05, 0.97, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="orange", weight='bold',bbox=props) 


    
    if not flag_ax:
        plt.show()   

In [ ]:
def GetFluxes2StarsTwoBands(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    May plot some calexp cutout for checking
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt"])
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])



            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt]
            entrycount+=1

        
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)
            continue
      

        if idx%10 == 0:
            if cutoutb1 is not None and cutoutb2 is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(row_pair_b1,cutoutb1,ax=ax1)
                plotcutout(row_pair_b2,cutoutb2,ax=ax2)
                plt.show()

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)
    
    return df_col_b1b2


In [ ]:
def GetFluxes2StarsTwoBandsNoCutout(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    NO NOT plot some calexp cutout contrary to above function
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt"])
    
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            #row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            #row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 
            row_pair_b1  = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]  
            row_pair_b2  = df_selectedvisits_b2[df_selectedvisits_b2.visit == visit2].iloc[0]  

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]    
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
              
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])



            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt]

            entrycount+=1
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)
            continue

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # absolute relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)


    return df_col_b1b2

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

### Spectroscopy

In [ ]:
atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
#df_spec["PWV [mm]_x"]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8))
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
df_spec.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)
plt.tight_layout()

## Photometry

#### read list of all possible pairs

In [ ]:
df_pair = pd.read_csv("all_pairs.csv",index_col=0)
df_pair.head()

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_pair["sep"].hist(bins=50,ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")


#### Select the pair to study

In [ ]:
idx_pair = 10
objectname1 = int(df_pair["objidx1"].iloc[idx_pair])
objectname2 = int(df_pair["objidx2"].iloc[idx_pair])

In [ ]:
df_pair.iloc[idx_pair]

### Read All sources from all objects 

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
path_photom ="data/photom/sources_objectTable"
tract = 3864

#inputfilename ="objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069-tract3864.csv"
inputfilename = "objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146-tract3864.csv"
inputfullfilename= os.path.join(path_photom,inputfilename) 

df_allsources = pd.read_csv(inputfullfilename,index_col=0)


In [ ]:
df_allsources.head()

#### Select all sources/visits corresponding to object1/object2 respectively

In [ ]:
df_myselectedvisits1 = df_allsources[df_allsources.obj_index==objectname1] 	
df_myselectedvisits2 = df_allsources[df_allsources.obj_index==objectname2] 	

In [ ]:
df_myselectedvisits1.sort_values("visit",inplace=True)
df_myselectedvisits2.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits1.head()

In [ ]:
df_myselectedvisits2.head()

### Compute nightobs and Times (pd.to_datetime)

In [ ]:
df_myselectedvisits1["nightObs"] = df_myselectedvisits1.apply(lambda x: x['visit']//100_000 , axis=1)
df_myselectedvisits1["Time"] = pd.to_datetime(df_myselectedvisits1['obsStart'])

In [ ]:
pd.to_datetime(df_myselectedvisits2['obsStart']).to_frame()

In [ ]:
df_myselectedvisits2.loc[:,"nightObs"] = df_myselectedvisits2.apply(lambda x: x['visit']//100_000 , axis=1,result_type='expand')
df_myselectedvisits2.loc[:,"Time"] = pd.to_datetime(df_myselectedvisits2['obsStart']).to_frame()

### Split list by bands

In [ ]:
df_myselectedvisits_g1 = df_myselectedvisits1[df_myselectedvisits1.band == "g"]
df_myselectedvisits_r1 = df_myselectedvisits1[df_myselectedvisits1.band == "r"]
df_myselectedvisits_z1 = df_myselectedvisits1[df_myselectedvisits1.band == "z"]
df_myselectedvisits_y1 = df_myselectedvisits1[df_myselectedvisits1.band == "y"]

df_myselectedvisits_g2 = df_myselectedvisits2[df_myselectedvisits2.band == "g"]
df_myselectedvisits_r2 = df_myselectedvisits2[df_myselectedvisits2.band == "r"]
df_myselectedvisits_z2 = df_myselectedvisits2[df_myselectedvisits2.band == "z"]
df_myselectedvisits_y2 = df_myselectedvisits2[df_myselectedvisits2.band == "y"]

In [ ]:
df_myselectedvisits_z1[["nightObs","visit"]].head() 

In [ ]:
df_myselectedvisits_z2[["nightObs","visit"]].head() 

In [ ]:
# no 2023112100507 in z
df_myselectedvisits_y1[["nightObs","visit"]].head() 

In [ ]:
df_myselectedvisits_y2[["nightObs","visit"]].head() 

### Merge visits in same band/same calexp for object 1 and Object 2 

In [ ]:
df_y = df_myselectedvisits_y1.merge(df_myselectedvisits_y2, left_on='visit', right_on='visit',how="inner")
df_z = df_myselectedvisits_z1.merge(df_myselectedvisits_z2, left_on='visit', right_on='visit',how="inner")
df_g = df_myselectedvisits_g1.merge(df_myselectedvisits_g2, left_on='visit', right_on='visit',how="inner")
df_r = df_myselectedvisits_r1.merge(df_myselectedvisits_r2, left_on='visit', right_on='visit',how="inner")

In [ ]:
df_z["visit"].head(n=5)

In [ ]:
df_y["visit"].head(n=5)

### Distance in pixels

In [ ]:
df_y["dx"] = df_y['x_x'] - df_y['x_y']
df_y["dy"] = df_y['y_x'] - df_y['y_y']
df_y["dpix"] = np.sqrt(df_y["dx"]**2 + df_y["dy"]**2)

In [ ]:
df_y["dpix"].hist(bins=50)
plt.xlabel("distance in pixels")

## Distance in time between the pairs

In [ ]:
df_y["dt"] = (df_y['expMidptMJD_x'] - df_y['expMidptMJD_y'])*3600*24

In [ ]:
df_y["dt"].hist(bins=50)

In [ ]:
# convert timedelta into seconds
# thus it confirms the exposure is the same
df_y["dTime"] = pd.to_timedelta(df_y['Time_x'] - df_y['Time_y']).dt.seconds

In [ ]:
df_y["dTime"].hist(bins=50)

## Rename quantities supposed to be the same as same visit

In [ ]:
df_y["band"] = df_y["band_x"]
df_z["band"] = df_z["band_x"]
df_g["band"] = df_g["band_x"]
df_r["band"] = df_r["band_x"]

df_y["Time"] = df_y["Time_x"]
df_z["Time"] = df_z["Time_x"]
df_g["Time"] = df_g["Time_x"]
df_r["Time"] = df_r["Time_x"]

df_y["nightObs"] = df_y["nightObs_x"]
df_z["nightObs"] = df_z["nightObs_x"]
df_g["nightObs"] = df_g["nightObs_x"]
df_r["nightObs"] = df_r["nightObs_x"]

#df_y["visit"] = df_y["visit_x"]
#df_z["visit"] = df_z["visit_x"]
#df_g["visit"] = df_g["visit_x"]
#df_r["visit"] = df_r["visit_x"]

df_y["expMidptMJD"] = df_y["expMidptMJD_x"]
df_z["expMidptMJD"] = df_z["expMidptMJD_x"]
df_g["expMidptMJD"] = df_g["expMidptMJD_x"]
df_r["expMidptMJD"] = df_r["expMidptMJD_x"]

## Difference in magnitudes between the pairs

#### psf Magnitude

In [ ]:
df_y["dpsfMag"] = df_y["psfMag_x"] -  df_y["psfMag_y"]
df_z["dpsfMag"] = df_z["psfMag_x"] -  df_z["psfMag_y"]
df_g["dpsfMag"] = df_g["psfMag_x"] -  df_g["psfMag_y"]
df_r["dpsfMag"] = df_r["psfMag_x"] -  df_r["psfMag_y"]

In [ ]:
df_y["dpsfInstMag"] = df_y["psfInstMag_x"] -  df_y["psfInstMag_y"]
df_z["dpsfInstMag"] = df_z["psfInstMag_x"] -  df_z["psfInstMag_y"]
df_g["dpsfInstMag"] = df_g["psfInstMag_x"] -  df_g["psfInstMag_y"]
df_r["dpsfInstMag"] = df_r["psfInstMag_x"] -  df_r["psfInstMag_y"]

In [ ]:
df_y["dpsfMagErr"] = np.sqrt(df_y["psfMagErr_x"]**2 +  df_y["psfMagErr_y"]**2) 
df_z["dpsfMagErr"] = np.sqrt(df_z["psfMagErr_x"]**2 +  df_z["psfMagErr_y"]**2) 
df_g["dpsfMagErr"] = np.sqrt(df_g["psfMagErr_x"]**2 +  df_g["psfMagErr_y"]**2) 
df_r["dpsfMagErr"] = np.sqrt(df_r["psfMagErr_x"]**2 +  df_r["psfMagErr_y"]**2) 

In [ ]:
df_y["dpsfInstMagErr"] = np.sqrt(df_y["psfInstMagErr_x"]**2 +  df_y["psfInstMagErr_y"]**2) 
df_z["dpsfInstMagErr"] = np.sqrt(df_z["psfInstMagErr_x"]**2 +  df_z["psfInstMagErr_y"]**2) 
df_g["dpsfInstMagErr"] = np.sqrt(df_g["psfInstMagErr_x"]**2 +  df_g["psfInstMagErr_y"]**2) 
df_r["dpsfInstMagErr"] = np.sqrt(df_r["psfInstMagErr_x"]**2 +  df_r["psfInstMagErr_y"]**2) 

#### Aperture Magnitude

In [ ]:
df_y["dapFlux_35_0_calMag"] = df_y["apFlux_35_0_calMag_x"] - df_y["apFlux_35_0_calMag_y"]
df_z["dapFlux_35_0_calMag"] = df_z["apFlux_35_0_calMag_x"] - df_z["apFlux_35_0_calMag_y"]
df_g["dapFlux_35_0_calMag"] = df_g["apFlux_35_0_calMag_x"] - df_g["apFlux_35_0_calMag_y"]
df_r["dapFlux_35_0_calMag"] = df_r["apFlux_35_0_calMag_x"] - df_r["apFlux_35_0_calMag_y"]

In [ ]:
df_y["dapFlux_35_0_calMagErr"] = np.sqrt(df_y["apFlux_35_0_calMagErr_x"]**2 + df_y["apFlux_35_0_calMagErr_y"]**2)
df_z["dapFlux_35_0_calMagErr"] = np.sqrt(df_z["apFlux_35_0_calMagErr_x"]**2 + df_z["apFlux_35_0_calMagErr_y"]**2)
df_g["dapFlux_35_0_calMagErr"] = np.sqrt(df_g["apFlux_35_0_calMagErr_x"]**2 + df_g["apFlux_35_0_calMagErr_y"]**2)
df_r["dapFlux_35_0_calMagErr"] = np.sqrt(df_r["apFlux_35_0_calMagErr_x"]**2 + df_r["apFlux_35_0_calMagErr_y"]**2)

In [ ]:
df_y["dapFlux_50_0_calMag"] = df_y["apFlux_50_0_calMag_x"] - df_y["apFlux_50_0_calMag_y"]
df_z["dapFlux_50_0_calMag"] = df_z["apFlux_50_0_calMag_x"] - df_z["apFlux_50_0_calMag_y"]
df_g["dapFlux_50_0_calMag"] = df_g["apFlux_50_0_calMag_x"] - df_g["apFlux_50_0_calMag_y"]
df_r["dapFlux_50_0_calMag"] = df_r["apFlux_50_0_calMag_x"] - df_r["apFlux_50_0_calMag_y"]

In [ ]:
df_y["dapFlux_50_0_calMagErr"] = np.sqrt(df_y["apFlux_50_0_calMagErr_x"]**2 + df_y["apFlux_50_0_calMagErr_y"]**2)
df_z["dapFlux_50_0_calMagErr"] = np.sqrt(df_z["apFlux_50_0_calMagErr_x"]**2 + df_z["apFlux_50_0_calMagErr_y"]**2)
df_g["dapFlux_50_0_calMagErr"] = np.sqrt(df_g["apFlux_50_0_calMagErr_x"]**2 + df_g["apFlux_50_0_calMagErr_y"]**2)
df_r["dapFlux_50_0_calMagErr"] = np.sqrt(df_r["apFlux_50_0_calMagErr_x"]**2 + df_r["apFlux_50_0_calMagErr_y"]**2)

### Relative magnitude difference

In [ ]:
df_y["d_psfMag"] = df_y["dpsfMag"] - df_y["dpsfMag"].mean()
df_z["d_psfMag"] = df_z["dpsfMag"] - df_z["dpsfMag"].mean()
df_g["d_psfMag"] = df_g["dpsfMag"] - df_g["dpsfMag"].mean()
df_r["d_psfMag"] = df_r["dpsfMag"] - df_r["dpsfMag"].mean()

In [ ]:
df_y["d_psfInstMag"] = df_y["dpsfInstMag"] - df_y["dpsfInstMag"].mean()
df_z["d_psfInstMag"] = df_z["dpsfInstMag"] - df_z["dpsfInstMag"].mean()
df_g["d_psfInstMag"] = df_g["dpsfInstMag"] - df_g["dpsfInstMag"].mean()
df_r["d_psfInstMag"] = df_r["dpsfInstMag"] - df_r["dpsfInstMag"].mean()

In [ ]:
df_y["d_apFlux_35_0_calMag"] = df_y["dapFlux_35_0_calMag"] - df_y["dapFlux_35_0_calMag"].mean()
df_z["d_apFlux_35_0_calMag"] = df_z["dapFlux_35_0_calMag"] - df_z["dapFlux_35_0_calMag"].mean()
df_g["d_apFlux_35_0_calMag"] = df_g["dapFlux_35_0_calMag"] - df_g["dapFlux_35_0_calMag"].mean()
df_r["d_apFlux_35_0_calMag"] = df_r["dapFlux_35_0_calMag"] - df_r["dapFlux_35_0_calMag"].mean()

In [ ]:
df_y["d_apFlux_50_0_calMag"] = df_y["dapFlux_50_0_calMag"] - df_y["dapFlux_50_0_calMag"].mean()
df_z["d_apFlux_50_0_calMag"] = df_z["dapFlux_50_0_calMag"] - df_z["dapFlux_50_0_calMag"].mean()
df_g["d_apFlux_50_0_calMag"] = df_g["dapFlux_50_0_calMag"] - df_g["dapFlux_50_0_calMag"].mean()
df_r["d_apFlux_50_0_calMag"] = df_r["dapFlux_50_0_calMag"] - df_r["dapFlux_50_0_calMag"].mean()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"psfFlux : AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_psfMag,yerr=df_g.dpsfMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_psfMag,yerr=df_r.dpsfMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_psfMag,yerr=df_z.dpsfMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_psfMag,yerr=df_y.dpsfMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.04,0.04)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; psfMag$ (mag)")
ax2.set_ylabel("$\Delta \; psfMag$ (mag)")
ax3.set_ylabel("$\Delta \; psfMag$ (mag)")
ax4.set_ylabel("$\Delta \; psfMag$ (mag)")
ax1.axhline(0,color="k")
ax2.axhline(0,color="k")
ax3.axhline(0,color="k")
ax4.axhline(0,color="k")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"psfInstFlux : AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_psfMag,yerr=df_g.dpsfMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_psfInstMag,yerr=df_r.dpsfMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_psfInstMag,yerr=df_z.dpsfMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_psfInstMag,yerr=df_y.dpsfMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.04,0.04)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; psfInstMag$ (mag)")
ax2.set_ylabel("$\Delta \; psfInstMag$ (mag)")
ax3.set_ylabel("$\Delta \; psfInstMag$ (mag)")
ax4.set_ylabel("$\Delta \; psfInstMag$ (mag)")
ax1.axhline(0,color="k")
ax2.axhline(0,color="k")
ax3.axhline(0,color="k")
ax4.axhline(0,color="k")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"Aperture Flux Ap35 : AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_apFlux_35_0_calMag,yerr=df_g.dapFlux_35_0_calMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_apFlux_35_0_calMag,yerr=df_r.dapFlux_35_0_calMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_apFlux_35_0_calMag,yerr=df_z.dapFlux_35_0_calMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_apFlux_35_0_calMag,yerr=df_y.dapFlux_35_0_calMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.06,0.06)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax1.axhline(0,color="k")
ax2.axhline(0,color="k")
ax3.axhline(0,color="k")
ax4.axhline(0,color="k")
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax2.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax3.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax4.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"Aperture Flux Ap50 : AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_apFlux_50_0_calMag,yerr=df_g.dapFlux_50_0_calMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_apFlux_50_0_calMag,yerr=df_r.dapFlux_50_0_calMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_apFlux_50_0_calMag,yerr=df_z.dapFlux_50_0_calMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_apFlux_50_0_calMag,yerr=df_y.dapFlux_50_0_calMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.06,0.06)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; Ap50 Mag$ (mag)")
ax2.set_ylabel("$\Delta \; Ap50 Mag$ (mag)")
ax3.set_ylabel("$\Delta \; Ap50 Mag$ (mag)")
ax4.set_ylabel("$\Delta \; Ap50 Mag$ (mag)")
ax1.axhline(0,color="k")
ax2.axhline(0,color="k")
ax3.axhline(0,color="k")
ax4.axhline(0,color="k")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

## Neighbouring visits

### band 1 : Group by NightObs and bands

In [ ]:
groups_obs_size1 = df_myselectedvisits1.groupby(by=["nightObs","band"]).size()

In [ ]:
groups_obs_size1

### band 1 : Group by NightObs and bands unstacked into a pandas dataframe

In [ ]:
df_groups_obs_size1 = groups_obs_size1.unstack()
df_groups_obs_size1

### band 1 : Group by NightObs and band and Visit

In [ ]:
groups_obs_visit1 = df_myselectedvisits1.groupby(by=["nightObs","band","visit"]).agg(time = ("Time","mean") ,psfMean = ("psfMag","mean"),apFlux_50_0_instFlux = ("apFlux_50_0_instFlux","mean"))
groups_obs_visit1 

### band 1 : Group by NightObs and visit and separate into different dataframe band by band

In [ ]:
groups_obs_bandg_visit1 = df_myselectedvisits_g1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandr_visit1 = df_myselectedvisits_r1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandz_visit1 = df_myselectedvisits_z1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandy_visit1 = df_myselectedvisits_y1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

In [ ]:
groups_obs_bandg_visit2 = df_myselectedvisits_g2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandr_visit2 = df_myselectedvisits_r2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandz_visit2 = df_myselectedvisits_z2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandy_visit2 = df_myselectedvisits_y2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

In [ ]:
groups_obs_bandz_visit1.iloc[:5]

In [ ]:
groups_obs_bandz_visit2.iloc[:5]

## Process night-per night, band by band

We want to associate pairs of observations in different bands:

- visits in g with visits in r,z,y
- visits in r with visits in z,y
- visits in z with visits in y

### Process band g and r

In [ ]:
dfg_r = closestvisit(groups_obs_bandg_visit1,groups_obs_bandr_visit2)
#dfg_r.drop(labels="deltat", axis=1,inplace=True)
dfg_r.columns = ["nightObs_r","visit_r","time_r","dtsec_r"]
dfg_r.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfg_r.head()

In [ ]:
groups_obs_bandg_visit1.join(dfg_r)

### Process band g and z

In [ ]:
dfg_z = closestvisit(groups_obs_bandg_visit1,groups_obs_bandz_visit2)
#dfg_z.drop(labels="deltat", axis=1,inplace=True)
dfg_z.columns = ["nightObs_z","visit_z","time_z","dtsec_z"]
dfg_z.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfg_z.head(n=10)

### Process band g and y

In [ ]:
dfg_y = closestvisit(groups_obs_bandg_visit1,groups_obs_bandy_visit2)
#dfg_y.drop(labels="deltat", axis=1,inplace=True)
dfg_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]
dfg_y.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfg_y.head()

### Concatenation g with r,z,y

In [ ]:
dfg_rzy = pd.concat([groups_obs_bandg_visit1,dfg_r,dfg_z,dfg_y],axis=1)

In [ ]:
dfg_rzy.head() 

### Process band r and z

In [ ]:
dfr_z = closestvisit(groups_obs_bandr_visit1,groups_obs_bandz_visit2)
#dfr_z.drop(labels="deltat", axis=1,inplace=True)
dfr_z.columns = ["nightObs_z","visit_z","time_z","dtsec_z"]
dfr_z.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfr_z.head()

### Process band r and y

In [ ]:
dfr_y = closestvisit(groups_obs_bandr_visit1,groups_obs_bandy_visit2)
#dfr_y.drop(labels="deltat", axis=1,inplace=True)
dfr_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]
dfr_y.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfr_y.head(n=10)

### Concatenation of r with z,y

In [ ]:
dfr_zy = pd.concat([groups_obs_bandr_visit1,dfr_z,dfr_y],axis=1)

In [ ]:
dfr_zy.head()

### Process band z and y

In [ ]:
dfz_y = closestvisit(groups_obs_bandz_visit1,groups_obs_bandy_visit2)
#dfz_y.drop(labels="deltat", axis=1,inplace=True)
dfz_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]
dfz_y.dropna(axis=1,how="any",inplace=True)

In [ ]:
dfz_y.head(n=10)

### Concatenate z with y
-  (2023112100560,2023112100561) 	 	
-  (2023112700311,2023112700312) 

In [ ]:
dfz_y = pd.concat([groups_obs_bandz_visit1,dfz_y],axis=1)

In [ ]:
# select with the hour
dfz_y = dfz_y[np.abs(dfz_y.dtsec_y)<3600]
dfz_y.head(n=10)

## Fetch a pair of  stars

In [ ]:
df_z.head(n=10)[["nightObs","visit"]]

In [ ]:
df_y.head(n=10)[["nightObs","visit"]]

## Differential colors study (star 1 - star 2)

### Build color ratio in G and R band

### G and R band colors

In [ ]:
dfg_r = dfg_rzy[["time","nightObs_r","visit_r","time_r","dtsec_r"]] 
dfg_r.head()

#### Select pairs in g,r within 1 minute

In [ ]:
dfg_r_sel = dfg_r[np.abs(dfg_r["dtsec_r"])<60]
dfg_r_sel.head() 

# Loop over band pairs exposures nearby in time to calculate colors and relative colors of two stars 

In [ ]:
df_g.head()

In [ ]:
#df_col_gr = GetFluxes2StarsTwoBands("g","r",dfg_r_sel,df_g,df_r)
df_col_gr = GetFluxes2StarsTwoBandsNoCutout("g","r",dfg_r_sel,df_g,df_r)

In [ ]:
df_col_gr.head()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")
df_col_gr.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c="b",lw=0.0,grid=True,label="rel. color difference G -R ")
ax2.errorbar(df_col_gr.time1,df_col_gr.apcol12_s12,yerr=df_col_gr.apcol12Err_s12,color="b",ecolor="k",fmt="o")
ax2.set_xlabel("time1")
ax2.xaxis.set_major_formatter(date_form)

tmin = df_col_gr["time1"].min()
tmax = df_col_gr["time1"].max()
dt = tmax-tmin
tmin = tmin-dt/10
tmax = tmax+dt/10

ax2.set_xlim(tmin,tmax)
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(G-R)_1-(G-R)_2|$")
title = f"objects {objectname1},{objectname2} : " + suptitle
plt.suptitle(title,y=0.9)

### G - Y bands

In [ ]:
dfg_y = dfg_rzy[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfg_y.head()

#### select close measurement in time

In [ ]:
dfg_y_sel = dfg_y[np.abs(dfg_y["dtsec_y"]<3600)]

In [ ]:
dfg_y_sel.head()

In [ ]:
#df_col_gy = GetFluxes2StarsTwoBands("g","y",dfg_y_sel,df_g,df_y)
df_col_gy = GetFluxes2StarsTwoBandsNoCutout("g","y",dfg_y_sel,df_g,df_y)

In [ ]:
df_col_gy.head()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")
df_col_gy.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c="b",lw=0.0,grid=True,label="rel. color difference G - Y ")
ax2.errorbar(df_col_gy.time1,df_col_gy.apcol12_s12,yerr=df_col_gy.apcol12Err_s12,color="b",ecolor="k",fmt="o")
ax2.set_xlabel("time1")
ax2.xaxis.set_major_formatter(date_form)

tmin = df_col_gy["time1"].min()
tmax = df_col_gy["time1"].max()
dt = tmax-tmin
tmin = tmin-dt/10
tmax = tmax+dt/10

ax2.set_xlim(tmin,tmax)
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35 : $|(G-Y)_1-(G-Y)_2|$")
title = f"objects {objectname1},{objectname2} : " + suptitle
plt.suptitle(title,y=0.9)

### R - Y bands

In [ ]:
dfr_y = dfr_zy[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfr_y.head()

In [ ]:
dfr_y_sel = dfr_y[np.abs(dfr_y["dtsec_y"])<3600]

In [ ]:
#df_col_ry = GetFluxes2StarsTwoBands("r","y",dfr_y_sel,df_r,df_y)
df_col_ry = GetFluxes2StarsTwoBandsNoCutout("r","y",dfr_y_sel,df_r,df_y)

In [ ]:
df_col_ry.head() 

In [ ]:
df_col_ry = df_col_ry[np.abs(df_col_ry["dt"])<3600]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")
df_col_ry.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c="b",lw=0.0,grid=True,label="rel. color difference R -Y ")
ax2.errorbar(df_col_ry.time1,df_col_ry.apcol12_s12,yerr=df_col_ry.apcol12Err_s12,color="b",ecolor="k",fmt="o")
ax2.set_xlabel("time1")
ax2.xaxis.set_major_formatter(date_form)

tmin = df_col_gy["time1"].min()
tmax = df_col_gy["time1"].max()
dt = tmax-tmin
tmin = tmin-dt/10
tmax = tmax+dt/10

ax2.set_xlim(tmin,tmax)

ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(R-Y)_1-(R-Y)_2|$")
title = f"objects {objectname1},{objectname2} : " + suptitle
plt.suptitle(title,y=0.9)

### Z - Y bands

In [ ]:
#dfz_y = dfz_y[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfz_y.head()

In [ ]:
dfz_y_sel = dfz_y[np.abs(dfz_y["dtsec_y"])<3600]

In [ ]:
#dfz_y_sel

In [ ]:
#df_col_zy = GetFluxes2StarsTwoBands("z","y",dfz_y_sel,df_z,df_y)
df_col_zy = GetFluxes2StarsTwoBandsNoCutout("z","y",dfz_y_sel,df_z,df_y)

In [ ]:
df_col_zy = df_col_zy[np.abs(df_col_zy["dt"])<3600]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%dT%H")
fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")
df_col_zy.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c="b",lw=0.0,grid=True,label="rel. color difference Z - Y")
ax2.errorbar(df_col_zy.time1,df_col_zy.psfcol12_s12,yerr=df_col_zy.psfcol12Err_s12,color="b",ecolor="k",fmt="o")
ax2.set_xlabel("time1")
ax2.xaxis.set_major_formatter(date_form)

tmin = df_col_zy["time1"].min()
tmax = df_col_zy["time1"].max()
dt = tmax-tmin
tmin = tmin-dt/10
tmax = tmax+dt/10

ax2.set_xlim(tmin,tmax)
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax2.legend(loc="upper left")
ax2.set_ylabel("PSF : $|(Z-Y)_1-(Z-Y)_2|$")
title = f"objects {objectname1},{objectname2} : " + suptitle
plt.suptitle(title,y=0.9)

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%dT%H")
fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")
df_col_zy.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c="b",lw=0.0,grid=True,label="rel. color difference Z - Y")
ax2.errorbar(df_col_zy.time1,df_col_zy.apcol12_s12,yerr=df_col_zy.apcol12Err_s12,color="b",ecolor="k",fmt="o")
ax2.set_xlabel("time1")
ax2.xaxis.set_major_formatter(date_form)

tmin = df_col_zy["time1"].min()
tmax = df_col_zy["time1"].max()
dt = tmax-tmin
tmin = tmin-dt/10
tmax = tmax+dt/10

ax2.set_xlim(tmin,tmax)
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35 : $|(Z-Y)_1-(Z-Y)_2|$")
title = f"objects {objectname1},{objectname2} : " + suptitle
plt.suptitle(title,y=0.9)